In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def create_bins(data, increment):
    max_val = abs(data).max()
    bin_edges = list(np.arange(0, max_val, increment))
    negative_bins = [-i for i in bin_edges][::-1]
    bin_edges = negative_bins + [0] + bin_edges
    return bin_edges

def group_tail_bins(bins, data_counts, mid_points, correlations):
    grouped_bins = []
    grouped_data_counts = []
    grouped_mid_points = []
    grouped_correlations = []
    
    i = 0
    while i < len(bins) - 1:
        if data_counts[i] >= min_data_points:
            grouped_bins.append((bins[i], bins[i+1]))
            grouped_data_counts.append(data_counts[i])
            grouped_mid_points.append(mid_points[i])
            grouped_correlations.append(correlations[i])
            i += 1
        else:
            start = i
            total_count = 0
            while i < len(bins) - 1 and data_counts[i] < min_data_points:
                total_count += data_counts[i]
                i += 1
            grouped_bins.append((bins[start], bins[i]))
            grouped_data_counts.append(total_count)
            grouped_mid_points.append(np.mean(mid_points[start:i]))
            grouped_correlations.append(np.nanmean(correlations[start:i]))
            
    return grouped_bins, grouped_data_counts, grouped_mid_points, grouped_correlations

# Read data from .xlsx file
file_path = '~/FRM344/project_data.xlsx'
original_df = pd.read_excel(file_path, index_col=0)
original_df = original_df.apply(pd.to_numeric, errors='coerce')

# Calculate returns
returns_df = original_df.pct_change().dropna()

covid_start_date = '2020-03-06'
pre_covid_df = returns_df[returns_df.index < covid_start_date]
post_covid_df = returns_df[returns_df.index >= covid_start_date]

increment = 0.0075
min_data_points = 15

datasets = {
    'Pre-COVID': pre_covid_df,
    'Post-COVID': post_covid_df,
    'Whole Period': returns_df
}

all_results_dict = {}

for period, data in datasets.items():
    results_dict = {}
    bins = create_bins(data['S&P 500'], increment)
    
    for asset in data.columns:
        if asset == 'S&P 500':  # Skip S&P 500 since it's our reference
            continue

        mid_points = []
        data_counts = []
        correlations = []

        for i in range(len(bins) - 1):
            mask = (data['S&P 500'] >= bins[i]) & (data['S&P 500'] < bins[i + 1])
            subset = data[mask]
            mid_point = (bins[i] + bins[i + 1]) / 2
            corr = subset[asset].corr(subset['S&P 500'])

            mid_points.append(mid_point)
            data_counts.append(len(subset))
            correlations.append(corr)

        grouped_bins, grouped_data_counts, grouped_mid_points, grouped_correlations = group_tail_bins(
            bins, data_counts, mid_points, correlations)

        df_asset = pd.DataFrame({
            'Bin (Return Range)': [f"[{grouped_bins[i][0]}%, {grouped_bins[i][1]}%]" for i in range(len(grouped_bins))],
            'Mid-point': grouped_mid_points,
            'Number of Data Points': grouped_data_counts,
            'Correlation with S&P 500': grouped_correlations
        })
    
        results_dict[asset] = df_asset

    all_results_dict[period] = results_dict

for asset_name in original_df.columns:
    if asset_name == 'S&P 500':  # Skip S&P 500 since it's our reference
        continue

    plt.figure(figsize=(10, 6))

    for period, color in zip(['Pre-COVID', 'Post-COVID', 'Whole Period'], ['green', 'red', 'grey']):
        test_asset = all_results_dict[period][asset_name]
        if 'Mid-point' in test_asset.columns and 'Correlation with S&P 500' in test_asset.columns:
            plt.plot(test_asset['Mid-point'], test_asset['Correlation with S&P 500'], marker='o', linestyle='-', color=color, label=period)

    plt.title(f"Conditional Correlation vs Mid-point for {asset_name}")
    plt.xlabel("Mid-point")
    plt.ylabel("Conditional Correlation with S&P 500")
    plt.legend()
    plt.grid(True)

    formatted_asset_name = ''.join(e for e in asset_name if e.isalnum())
    filename = f"{formatted_asset_name}_corr.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()

# Create the summary DataFrame
summary_data = []

for asset_name in original_df.columns:
    if asset_name == 'S&P 500':
        continue

    asset_summary = {'asset': asset_name}
    for period in ['Pre-COVID', 'Post-COVID', 'Whole Period']:
        test_asset = all_results_dict[period][asset_name]
        avg_corr = test_asset['Correlation with S&P 500'].mean()
        asset_summary[period] = avg_corr

    summary_data.append(asset_summary)

summary_df = pd.DataFrame(summary_data)
print(summary_df)


/usr/lib/python3.11/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/tmp/ipykernel_3512/2129583109.py:35: RuntimeWarning: Mean of empty slice
  grouped_correlations.append(np.nanmean(correlations[start:i]))
/usr/lib/python3.11/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/tmp/ipykernel_3512/2129583109.py:35: RuntimeWarning: Mean of empty slice
  grouped_correlations.append(np.nanmean(correlations[start:i]))
/usr/lib/python3.11/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for s

         asset  Pre-COVID  Post-COVID  Whole Period
0      AAPL US  -0.045321    0.191511      0.159240
1      TSLA US   0.028414    0.088164     -0.012813
2       WMT US   0.109778    0.058395      0.106213
3         Gold   0.133682    0.192202      0.132757
4  Brent Crude   0.146194    0.094827      0.033412
5         Corn   0.035007   -0.013622      0.031359
6         REIT   0.234442    0.194538      0.176136
7         Bond   0.153298    0.204551      0.108464
